In [56]:
from Mesh import *
from sklearn.neighbors import KDTree
import numpy as np
import datetime

mesh = Mesh()
mesh.read_from_off('data/1.off')
# for i in range(10):
#     e = mesh.edges[i]
#     print(e.index,e, e.nh, e.ph)
mesh.set_face_normal()
mesh.set_vertex_normal()

v = []
n = []
for i in mesh.vertices:
    v.append(i.p)
    n.append(i.normal)
fpfh = FPFH(v, n)
pfh = PFH(v, n)

[array([ 404, 1306, 1399, 1194,  259, 1195, 1196, 1218, 1455, 1304,  261,
       1452, 1307,  766,    0,  576, 1189,  765,   86,  275,  516, 1217,
         49,  396,  514,  767,  768,  513,  515,   15])
 array([1277,  573,  571,   75,   76,  733,  572,  856,  570,  890,  158,
       1216, 1215,  575,  574, 1305,  398, 1218, 1445, 1455,    1,  577,
        578, 1188,  256,  261,  576, 1189, 1121, 1093,  115, 1069,  204,
        211,  283,  888])
 array([ 508, 1223, 1224, 1225,  497,  716, 1239,   71,  507,  502,  506,
        504,   72,  479,  521,    5,  522, 1244,  464,  423,  465,  466,
       1204,    2,  496,  500,   14,    3,  499,  264,    6,  445,  498,
        234])
 ...
 array([ 305,  243,  285, 1164,  302, 1254,  286, 1255, 1453, 1253, 1279,
       1235, 1179, 1405, 1250,  247, 1162, 1463, 1466,  248, 1177, 1168,
       1165,  289, 1269, 1173, 1176,  122, 1167, 1243, 1166, 1267])
 array([ 740, 1460, 1321,  473, 1465, 1468, 1458,  332, 1072, 1464,  330,
        376,  470,    4

883 112
884 123
885 148
886 93
887 145
888 147
889 175
890 182
891 178
892 131
893 114
894 113
895 132
896 129
897 125
898 136
899 129
900 123
901 163
902 113
903 122
904 117
905 129
906 106
907 110
908 127
909 143
910 146
911 118
912 157
913 158
914 51
915 170
916 184
917 106
918 133
919 109
920 135
921 119
922 130
923 231
924 226
925 225
926 233
927 125
928 143
929 184
930 170
931 162
932 177
933 178
934 182
935 149
936 170
937 180
938 158
939 156
940 136
941 158
942 162
943 180
944 160
945 155
946 107
947 107
948 110
949 202
950 212
951 184
952 160
953 182
954 96
955 100
956 82
957 89
958 91
959 156
960 140
961 152
962 151
963 168
964 186
965 201
966 182
967 48
968 49
969 49
970 109
971 111
972 109
973 173
974 198
975 184
976 199
977 190
978 104
979 131
980 142
981 154
982 168
983 138
984 182
985 171
986 165
987 172
988 182
989 208
990 213
991 198
992 198
993 184
994 188
995 198
996 201
997 191
998 103
999 95
1000 96
1001 99
1002 117
1003 108
1004 188
1005 147
1006 160
1007 125
1008

678 25
679 26
680 26
681 25
682 27
683 24
684 27
685 27
686 25
687 26
688 33
689 31
690 40
691 25
692 34
693 32
694 35
695 43
696 41
697 30
698 37
699 30
700 39
701 47
702 48
703 46
704 33
705 30
706 25
707 33
708 32
709 35
710 34
711 35
712 34
713 34
714 39
715 37
716 37
717 31
718 18
719 27
720 28
721 34
722 35
723 32
724 30
725 26
726 30
727 28
728 31
729 29
730 31
731 31
732 35
733 31
734 31
735 31
736 29
737 31
738 31
739 28
740 34
741 30
742 27
743 38
744 46
745 40
746 31
747 29
748 27
749 30
750 28
751 48
752 43
753 49
754 47
755 44
756 40
757 35
758 43
759 34
760 45
761 16
762 18
763 29
764 28
765 28
766 27
767 28
768 35
769 32
770 29
771 29
772 29
773 33
774 23
775 30
776 30
777 23
778 28
779 28
780 30
781 32
782 30
783 28
784 33
785 42
786 39
787 32
788 40
789 32
790 42
791 36
792 31
793 26
794 30
795 27
796 24
797 24
798 26
799 23
800 26
801 35
802 35
803 27
804 27
805 28
806 27
807 25
808 27
809 32
810 29
811 28
812 34
813 29
814 26
815 34
816 33
817 26
818 28
819 27
820 26

In [55]:
def FPFH(vertices, normals, r = 0.1):
    # default parameters
    sub = 11
    bins = 11 * 3
    l = len(vertices)
    
    vertices_xyz = []
    for v in vertices:
        vertices_xyz.append(v.data)
    # build kd tree
    tree = KDTree(vertices_xyz, leaf_size=2)  
    # find neighbours within distance of k for every vertex 
    ind = tree.query_radius(vertices_xyz, r, count_only = False, return_distance = False)
    print(ind)
    # FPFH(p) = SPF(p) + 1/k * sigma( 1/w(k) * SPF(p(k)) )
    spf = np.zeros([l, bins])
    for i in range(l):
        pi = vertices[i]
        ni = normals[i]
        u = ni
        for j in ind[i]:
            if j == i:
                continue
            #count u v w for pi and its neighbours (bj)
            pj = vertices[j]
            p_p = pj - pi
            v = Vector3D.cross_product(p_p, u)
            w = Vector3D.cross_product(u, v)
            nj = normals[j]
            
            #count alpha phi theta ..
            alpha = Vector3D.dot_product(v, nj)
            phi = Vector3D.dot_product(u, p_p) * (1 / p_p.length)
            theta = math.atan2(Vector3D.dot_product(w, nj), Vector3D.dot_product(u, nj)) / math.pi
            
            #encode alpha phi theta into 33 bins, sub = 11 (default)
            index_a = int((alpha + 1) * sub / 2)
            if index_a > sub - 1:
                index_a = sub - 1
            index_b = int((phi + 1) * sub / 2)
            if index_b > sub - 1:
                index_b = sub - 1
            index_b += sub
            index_c = int((theta + 1) * sub / 2)
            if index_c > sub - 1:
                index_c = sub - 1
            index_c += sub * 2
            spf[i][index_a] += 1
            spf[i][index_b] += 1
            spf[i][index_c] += 1
    
    
    # FPFH(p) = SPF(p) + 1/k * sigma( 1/w(k) * SPF(p(k)) )
    fpfh = np.zeros([l, bins])        
    for i in range(l):
        fpfh[i] += spf[i][0]
        pi = vertices[i]
        k = len(ind[i]) - 1
        for j in ind[i]:
            if j == i:
                continue
            pj = vertices[j]
            w = (pi - pj).length
            fpfh[i] += spf[j] / w / k
    return fpfh

def PFH(vertices, normals, r = 0.1):
    # default parameters
    sub = 5
    bins = 5 ** 3
    l = len(vertices)
    
    vertices_xyz = []
    for v in vertices:
        vertices_xyz.append(v.data)
    # build kd tree
    tree = KDTree(vertices_xyz, leaf_size=2)  
    ind = tree.query_radius(vertices_xyz, r * 2, count_only = False, return_distance = False)
    
    
    #store the index of pi and pj in dict 
    apt = dict()
    for i in range(l):
        print(i, len(ind[i]))
        pi = vertices[i]
        ni = normals[i]
        u = ni
        for j in ind[i]:
            if i == j:
                continue
            #count u v w for pi and its neighbours (bj)
            pj = vertices[j]
            p_p = pj - pi
            v = Vector3D.cross_product(p_p, u)
            w = Vector3D.cross_product(u, v)
            nj = normals[j]
            
            #count alpha phi theta ..
            alpha = Vector3D.dot_product(v, nj)
            phi = Vector3D.dot_product(u, p_p) * (1 / p_p.length)
            theta = math.atan2(Vector3D.dot_product(w, nj), Vector3D.dot_product(u, nj)) / math.pi
            
            #encode alpha phi theta into 125 bins, sub = 5 (default)
            index_a = int((alpha + 1) * sub / 2)
            if index_a > sub - 1:
                index_a = sub - 1
            index_b = int((phi + 1) * sub / 2)
            if index_b > sub - 1:
                index_b = sub - 1
            index_c = int((theta + 1) * sub / 2)
            if index_c > sub - 1:
                index_c = sub - 1
            index = index_a * sub * sub + index_b * sub + index_c
            
            #store the index of pi and pj in dict 
            code = "{}_{}".format(i,j)
            apt[code] = index
            
    # find neighbours within distance of k for every vertex 
    ind = tree.query_radius(vertices_xyz, r, count_only = False, return_distance = False)
    pfh = np.zeros([l, bins])
    for ii in range(l):
        print(ii, len(ind[ii]))
        for i in ind[ii]:
            pi = vertices[i]
            ni = normals[i]
            u = ni
            for j in ind[ii]:
                if i == j:
                    continue
                code = "{}_{}".format(i,j)
                index = apt[code]
                pfh[ii][index] += 1
    return pfh

[[  0.           0.           0.         ...   1.77541037   4.79728244
   23.68028969]
 [  0.           0.           0.         ...   0.78352181   3.08330857
   45.68090037]
 [  0.           0.           0.         ...   4.68134296   5.68681896
  158.39489783]
 ...
 [  0.           0.           0.         ...   0.69473471   3.35286673
  181.24916565]
 [  0.           0.           0.         ...   7.25959207   6.72485349
  144.51388587]
 [  0.           0.           0.         ...   6.97613771   6.54765951
  165.32343145]]
